In [1]:
import ROOT
import root_numpy as rnpy
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn
import pickle

Welcome to ROOTaaS 6.06/08


In [4]:
d1 = pd.DataFrame(rnpy.root2array("/mnt/t3nfs01/data01/shome/jpata/btv/gc/TagVarExtractor/GCa08e5e237323/TT_TuneCUETP8M1_13TeV-powheg-pythia8/job_0_out.root",
                                 treename = "tagVars/ttree", start = 0, stop = 20))

In [6]:
d2 = pd.DataFrame(rnpy.root2array("/mnt/t3nfs01/data01/shome/jpata/btv/gc/TagVarExtractor/GCa08e5e237323/TT_TuneCUETP8M1_13TeV-powheg-pythia8/job_0_out.root",
                                 treename = "tagVars/ttree_track", start = 0, stop = 20))

/swshare/anaconda/lib/python2.7/site-packages/root_numpy/_tree.py:209: RuntimeWarning: ignoring duplicate branch named 'Track_nHitPixel'
  warn_missing_tree)


In [12]:
d1['track_data'] = pd.np.empty((len(d1.index), 0)).tolist()

In [10]:
d2["Track_jetIndex"]

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     1
8     1
9     1
10    1
11    1
12    1
13    2
14    2
15    2
16    2
17    2
18    2
19    3
Name: Track_jetIndex, dtype: uint64

In [14]:
# iterate over the track list to join jets with the tracks belonging to them
for irow, row in d2.iterrows():
    # these are the track data of the current track:
    tracks = row[["Track_pt", "Track_eta", "Track_phi", "Track_dxy", "Track_dz", "Track_IP", "Track_IP2D", "Track_length"]].as_matrix()
    jet_index = int(row["Track_jetIndex"])
    table_index = d1.loc[d1['Jet_jetIndex'] == jet_index].index[0]
    
    # append the tracks data to the matching jet in the main table
    d1['track_data'][table_index].append(tracks)

In [20]:
# now divide the jets and put them in separate lists, according to their flavour
jets_b = []
jets_l = []
jets_c = []

# iterate over the jet list, with already matched tracks
for irow, row in d1.iterrows():
    flavour = int(row["Jet_flavour"])
    
    # select the right list this jet belongs to
    if abs(flavour) == 5:
        jets = jets_b
    elif abs(flavour) == 4:
        jets = jets_c
    else:
        jets = jets_l
        
    # add the new jet to the list
    jets += [(row["Jet_pt"], row["Jet_eta"], row["Jet_phi"], row["Jet_mass"], flavour, row["track_data"])]

In [113]:
# save the thus processed data
with open('./jets_b.dat', 'wb') as f:
    pickle.dump(jets_b, f)
with open('./jets_l.dat', 'wb') as f:
    pickle.dump(jets_b, f)
with open('./jets_c.dat', 'wb') as f:
    pickle.dump(jets_b, f)

In [114]:
# try to load them back
with open('./jets_b.dat', 'rb') as f:
    test = pickle.load(f)

In [25]:
test = pd.DataFrame(jets_b, columns = ['Jet_pt', 'Jet_eta', 'Jet_phi', 'Jet_mass', 'flavor', 'track_data'])

In [38]:
len(test.loc[0]['track_data'])

12